# Attention Dot Model

In [ ]:
## LOADING THE REQUIRED LIBRARIES 

import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import tensorflow as tf
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
from tqdm import tqdm

## Loading Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## LOADDING THE PROCESSED DATASET

df= pd.read_csv("/content/drive/MyDrive/ColabNotebooks/cs2/processed_data.csv")
df.columns = ["enc_input","dec_input","y"] 
df["dec_output"] = df.dec_input

In [ ]:
## THE INPUTS TO THE DECODER REQUIRES SPECIAL TOKENS FOR THE START AND THE END SO WE ARE GOING TO USE 
## <start> AS BEGINING TOKEN
## <end>  AS END TOKEN

df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>" 

## Splitting And Sampling around 100k datapoints

---
##### THE TOTAL DATASET HAS 500K DATAPOINTS WHICH WILL TAKE MUCH HIGHER TRAINING TIME. THEREFORE I AM SAMPLING ONE-FIFTH OF THE TOTAL DATASET



In [ ]:
df_sampled = pd.concat((df[df.y==1].sample(frac= 0.2,random_state=1),df[df.y==2]))

In [ ]:
## ONCE THE DATA IS SAMPLED WE ARE SPLITTIND THE DATA IN TO TRAIN AND TEST
df_train ,df_val = train_test_split(df_sampled,test_size=0.2,random_state = 3, stratify = df_sampled.y )

In [ ]:
## IN THE COLUMN WHICH HAS DECODER INPUTS ADDING "<end>" TOKEN TO BE LEARNED BY THE TOKENIZER
df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"

In [ ]:
## HERE I AM SAMPLING 1000 POINTS FROM THE DATAFRAME AS TEST DATA WHICH ARE NOT PRESEENT IN THE TRAIN AND VALIDAION DATA
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values if x not in df_sampled.index.values]),1000,replace= False,)]

## Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
## TOKENIZER FOR ENCODER INPUT
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))

In [ ]:
# TOKENIZER FOR DECODER INPUT
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

## Dataset Loader

In [ ]:
## THIS CLASS CONVERTS TEXT DATA TO INTEGER SEQUENCES AND RETURNS THE PADDED SEQUENCES
class Dataset :
    def __init__(self, data , tk_inp ,tk_out, max_len):
        ## SETTING THE REQUIRED ATTRIBUTES
        self.encoder_inp = data["enc_input"].apply(str).values
        self.decoder_inp = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp
        self.tk_out = tk_out
        self.max_len = max_len
        
    def __getitem__(self,i):
        # INPUT SEQUENCES
        self.encoder_seq = self.tk_inp.texts_to_sequences([self.encoder_inp[i]])
        # DECODER INPUT SEQUENCES 
        self.decoder_inp_seq = self.tk_out.texts_to_sequences([self.decoder_inp[i]])
        # DECODER INPUT SEQUENCES
        self.decoder_out_seq = self.tk_out.texts_to_sequences([self.decoder_out[i]])
        
        # PADDING THE ENCODER INPUT SEQUENCES
        self.encoder_seq = pad_sequences(self.encoder_seq, padding="post",maxlen = self.max_len)
        # PADDING THE DECODER INPUT SEQUENCES
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, padding="post",maxlen = self.max_len)
        # PADDING DECODER OUTPUT SEQUENCES
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq ,padding="post", maxlen = self.max_len)
        ##  RETURNING THE ENCODER INPUT , DECODER INPUT , AND DECODER OUTPUT
        return self.encoder_seq ,  self.decoder_inp_seq,  self.decoder_out_seq
    
    def __len__(self):
        # RETURN THE LEN OF INPUT ENDODER
        return len(self.encoder_inp)

In [ ]:
## THIS CLASS CONVERTES THE DATASET INTO THE REQUIRED BATCH SIZE

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self,batch_size,dataset):
        # INTIALIZING THE REQUIRED VARIABLES 
        self.dataset = dataset
        self.batch_size = batch_size
        self.totl_points = self.dataset.encoder_inp.shape[0]
        
    def __getitem__(self,i):
        # STATING THE START AND STOP VATIABLE CONTAINGING INDEX VALUES FOR EACH BATCH
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        # PLACEHOLDERS FOR BATCHED DATA
        batch_enc =[]
        batch_dec_input = []
        batch_dec_out =[]

        for j in range(start,stop): 
            
            a,b,c = self.dataset[j] 
            batch_enc.append(a[0]) 
            batch_dec_input.append(b[0])
            batch_dec_out.append(c[0]) 
        
        # Conveting list to array   
        batch_enc = (np.array(batch_enc)) 
        batch_dec_input = np.array(batch_dec_input)
        batch_dec_out = np.array(batch_dec_out)
        
        return [batch_enc , batch_dec_input],batch_dec_out
    
    def __len__(self):
        # Returning the number of batches
        return int(self.totl_points/self.batch_size)

###### WE ARE TAKING THE MAXIMUM LENGHT EQUAL TO 35 WHICH IS 99 PERCENTILE OF THE WORD LENGTH DISTRUBUTION

In [ ]:
# FORMING OBJECTS OF DATASET AND DATALOADER FOR TRAIN DATASET
train_dataset = Dataset(df_train,tk_inp,tk_out,35)
train_dataloader = Dataloader( batch_size = 512, dataset=train_dataset)

# FORMING OBJECTS OF DATASET AND DATALOADER FOR VALIDATION DATASET
val_dataset = Dataset(df_val , tk_inp,tk_out,35)
val_dataloader = Dataloader(batch_size=512 , dataset=val_dataset)

## Attention Model

In [ ]:
## LOADING THE TENSORFLOW LIBRARIES

from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
## DEFINING THE ENCODER LAYER AS A FUNCTION
class Encoder(tf.keras.layers.Layer):

    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''
    
    def __init__(self, vocab_size,emb_dims, enc_units, input_length,batch_size):
        super().__init__()
        # INITIALIZING THE REQUIRED VARIABLES
        self.batch_size=batch_size # BATHCH SIZE
        self.enc_units = enc_units # ENCODER UNITS

        # EMBEDDING LAYER
        self.embedding= layers.Embedding(vocab_size ,emb_dims) 
        # LSTM LAYER WITH RETURN SEQ AND RETURN STATES
        self.lstm = layers.LSTM(self.enc_units,return_state= True,return_sequences =  True) 

    def call(self, enc_input , states):
        
        # FORMING THE EMBEDDED VECTOR 
        emb = self.embedding(enc_input)
        # PASSING THE EMBEDDED VECTIO THROUGH LSTM LAYERS 
        enc_output,state_h,state_c = self.lstm(emb,initial_state=states)
        #RETURNING THE OUTPUT OF LSTM LAYER
        return enc_output,state_h,state_c 

    def initialize(self,batch_size):
      
        return tf.zeros(shape=(batch_size,self.enc_units)),tf.zeros(shape=(batch_size,self.enc_units))

In [ ]:
# THIS IS ATTNETION LAYER FOR DOT MODEL
class Attention(tf.keras.layers.Layer):
  
    '''THIS FUNCTION RETURNS THE CONTEXT VECTOR AND ATTENTION WEIGHTS (ALPHA VALUES)'''
    def __init__(self,units):
        super().__init__()
        # INITIALIZING THE NUMBER OF UNITS IN ATTENTION MODEL
        self.units =  units

    def call(self,enc_output,dec_state):
        # EXPANDING THE DIMENSION OF DECODER STATE  EG. FROM (16,32) TO (16,32,1)
        dec_state =  tf.expand_dims(dec_state,axis=-1)
        
        # MATRIX MULTIPLICATION OF ENCODER OUTPUT AND MODIFIED DECODER STATE
        # (16,32,1)*(16,13,32) = (16,13,1)
        score = tf.matmul(enc_output,dec_state)
        
        # APPLYING SOFTMAX TO THE AXIS 1
        # OUPUT SHAPE = (16,13,1)
        att_weights = tf.nn.softmax(score,axis=1)
        
        # CALCULATING THE CONTEXT VECTOR BY FIRST ELEMENTWISE MULTIPLICATION AND THEN ADDING THE AXIS 1
        # (16,13,1)*(16,13,32)=(16,13,32)
        context_vec  = att_weights* enc_output
        # (16,13,32) SUM AND REDUCE THE DIMENSION AT AXIS 1 => (16,32)
        context_vec = tf.reduce_sum(context_vec,axis=1)
        
        # RETURNING THE CONTEXT VECTOR AND ATTENTION WEIGHTS
        return context_vec,att_weights

In [ ]:
class Onestepdecoder(tf.keras.Model):
    '''THIS MODEL OUTPUTS THE RESULT OF DECODER FOR ONE TIME SETP GIVEN THE INPUT FOR PRECIOVE TIME STEP'''
    
    def __init__(self, vocab_size,emb_dims, dec_units, input_len,att_units,batch_size):
        super().__init__()
        # INTITALIZING THE REQUIRED VARIABLES
        
        # EMBEDDING LAYERS
        self.emb = layers.Embedding(vocab_size,emb_dims,input_length= input_len)
        # ATTENTION LAYER
        self.att = Attention(att_units)
        # LSTM LAYER
        self.lstm = layers.LSTM(dec_units,return_sequences=True,return_state=True)
        # DENSE LAYER
        self.dense = layers.Dense(vocab_size,activation="softmax")

    def call(self, encoder_output , input , state_h,state_c):
        # FORMING THE EMBEDDED VECTOR FOR THE WORD
        # (32,1)=>(32,1,12)
        emb = self.emb(input)

        dec_output,dec_state_h,dec_state_c = self.lstm(emb, initial_state = [state_h,state_c] )

        # GETTING THE CONTEXT VECTOR AND ATTENTION WEIGHTS BASED ON THE ENCODER OUTPUT AND  DECODER STATE_H
        context_vec,alphas = self.att(encoder_output,dec_state_h)
        
        # CONCATINATING THE CONTEXT VECTOR(BY EXPANDING DIMENSION) AND ENBEDDED VECTOR
        dense_input =  tf.concat([tf.expand_dims(context_vec,1),dec_output],axis=-1)
        
        # PASSING THE DECODER OUTPUT THROUGH DENSE LAYER WITH UNITS EQUAL TO VOCAB SIZE
        fc = self.dense(dense_input)
        
        # RETURNING THE OUTPUT
        return fc , dec_state_h , dec_state_c , alphas

In [ ]:
class Decoder(tf.keras.Model):
    '''THIS MODEL PERFORMS THE WHOLE DECODER OPERATION FOR THE COMPLETE SENTENCE'''
    def __init__(self, vocab_size,emb_dims, dec_units, input_len,att_units,batch_size):
        super().__init__()
        # INITIALIZING THE VARIABLES
        # LENGTH OF INPUT SENTENCE
        self.input_len = input_len
        # ONE STEP DECODER
        self.onestepdecoder = Onestepdecoder(vocab_size,emb_dims, dec_units, input_len,att_units,batch_size)

    def call(self,dec_input,enc_output,state_h,state_c):
        # THIS VATIABLE STORES THE VALUE OF STATE_H FOR THE PREVIOUS STATE
        current_state_h = state_h 
        current_state_c = state_c
        # THIS STORES THE DECODER OUTPUT FOR EACH TIME STEP
        pred = []
        # THIS STORED THE ALPHA VALUES
        alpha_values = []
        # FOR EACH WORD IN THE INPUT SENTENCE
        for i in range(self.input_len):
            
            # CURRENT WORD TO INPUT TO ONE STEP DECODER
            current_vec = dec_input[:,i]

            # EXPANDING THE DIMENSION FOR THE WORD
            current_vec = tf.expand_dims(current_vec,axis=-1)

            # PERFORMING THE ONE STEP DECODER OPERATION 
            dec_output,dec_state_h,dec_state_c,alphas = self.onestepdecoder(enc_output ,current_vec,current_state_h,current_state_c)

            #UPDATING THE CURRENT STATE_H
            current_state_h = dec_state_h
            current_state_c = dec_state_c

            #APPENDING THE DECODER OUTPUT TO "pred" LIST
            pred.append(dec_output)

            # APPENDING THE ALPHA VALUES
            alpha_values.append(alphas)
            
        # CONCATINATING ALL THE VALUES IN THE LIST
        output = tf.concat(pred,axis=1)
        # CONCATINATING ALL THE ALPHA VALUES IN THE LIST
        alpha_values = tf.concat(alpha_values,axis = -1)
        # RETURNING THE OUTPUT
        return output , alpha_values


In [ ]:
class encoder_decoder(tf.keras.Model):
    '''THIS MODEL COMBINES ALL THE LAYERS AND FORM IN ENCODER DECODER MODEL WITH ATTENTION MECHANISM'''
    def __init__(self,enc_vocab_size,enc_emb_dim,enc_units,enc_input_length,
             dec_vocab_size,dec_emb_dim,dec_units,dec_input_length ,att_units, batch_size):
        # INITAILIZING ALL VARIABLES
        super().__init__()
        # BATCH SIZE
        self.batch_size = batch_size
        # INITIALIZING ENCODER LAYER
        self.encoder = Encoder(enc_vocab_size, enc_emb_dim,enc_units, enc_input_length,batch_size)
        # INITALIZING DECODER LAYER
        self.decoder = Decoder(dec_vocab_size ,dec_emb_dim,dec_units,dec_input_length  ,att_units, batch_size)

    def call(self,data):
        # THE INPUT OF DATALOADER IS IN A LIST FORM FOR EACH BATCH IT GIVER TWO INPUTS
        # INPUT1 IS FOR ENCODER
        # INPUT2 IS FOR DECODER
        inp1 , inp2 = data
        # PASSING THE INPUT1 TO ENCODER LAYER
        enc_output, enc_state_h, enc_state_c = self.encoder(inp1,self.encoder.initialize(self.batch_size))
        # PASSING INPUT2 TO THE DECODER LAYER
        dec_output , alphas = self.decoder(inp2 , enc_output,enc_state_h,enc_state_c)
        # THE OUTPUT OF MODEL IS ONLY DECODER OUTPUT THE ALPHA VALUES ARE IGNORED HERE
        return dec_output



In [ ]:
# INITAILZING THE MODEL
model = encoder_decoder(enc_vocab_size=len(tk_inp.word_index)+1,
                         enc_emb_dim = 300,
                         enc_units=256,enc_input_length=35,
                         dec_vocab_size =len(tk_out.word_index)+1,
                         dec_emb_dim =300,
                         dec_units=256,
                         dec_input_length = 35,
                         
                         att_units=256,
                         batch_size=512)

In [ ]:
## DEFINING THE CALLBACKS
callback =[ tf.keras.callbacks.ModelCheckpoint( "/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_dot/besh.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001),
            tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_dot/logs/save",histogram_freq=1)
]
## STORING THE NUMBER OF STEPS IN ONE EPOCH FOR TRAIN AND VALIDATION DATASET
train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()
# COMPILING THE MODEL
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy')

In [ ]:
## FITTING THE MODEL
model.fit(train_dataloader, steps_per_epoch=train_steps,epochs= 50,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback)

In [ ]:
model.build([(512,35),(512,35)])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_dot/logs/save

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True)

In [ ]:
# LOADING THE WEIGHTS FOR BEST MODEL
model.load_weights("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_dot/besh.h5")

In [ ]:
## THIS FUNCTION IS USED IN THE INFERENCE TIME TO PREDICT THE RESULTS GIVEN THE INPUT TEXT

def predict(ita_text,model):
    '''THIS FUNCTION IS USED IN INFERENCE TIME WHICH GIVEN ANY SENTENCE IN ITALIAN OUTPUTS THE ENGLISH SENTENCE AND ALPHA VALUES'''
    # FORMING TOKENIZED SEQUENCES FOR INPUT SENTENCE
    seq = tk_inp.texts_to_sequences([ita_text])
    # PADDING THE SEQUENCES
    seq = pad_sequences(seq,maxlen = 20 , padding="post")
    # INITIALIZING THE STATES FOR INPUTING TO ENCODER
    state = model.layers[0].initialize(1)
    # GETTING THE ENCODED OUTPUT
    enc_output,state_h,state_c= model.layers[0](seq,state)
    # VARIABLE TO STORE PREDICTED SENTENCE
    pred = []
    # THIS VARIABLE STORES THE STATE TO BE INPUTED TO ONE STEP ENCODER
    input_state_h = state_h
    input_state_c = state_c
    # THIS VARIABLE STORES THE VECTOR TO VE INPUTED TO ONE STEP ENCODER
    current_vec = tf.ones((1,1))
    # THIS VARIABLE WILL STORE ALL THE ALPHA VALUES OUTPUTS
    alpha_values = []

    for i in range(20):
        # PASSING THE REQUIRED VARIABLE TO ONE STEP ENCODER LAYER
        fc , dec_state_h ,dec_state_c, alphas = model.layers[1].layers[0](enc_output , current_vec ,input_state_h ,input_state_c)
        #APPENDING THE ALPHA VALUES TO THE LIST "alpha_values"
        alpha_values.append(alphas)
         # UPDATING THE CURRENT VECTOR 
        current_vec = np.argmax(fc , axis = -1)
         # UPDATING THE INPUT STATE
        input_state_h = dec_state_h
        input_state_c = dec_state_c
        # GETTING THE ACTUAL WORDS FRO THE TOKENIZED INDEXES
        pred.append(tk_out.index_word[current_vec[0][0]])
        # IF THE WORD "<end>" COMES THE LOOP WILL BREAK
        if tk_out.index_word[current_vec[0][0]]=="<end>":
              break
    # JOINING THE PREDICTED WORDS
    pred_sent = " ".join(pred)
    # CONCATINATING ALL THE ALPHA VALUES
    alpha_values = tf.squeeze(tf.concat(alpha_values,axis=-1),axis=0)
    # RETURNING THE PREDICTED SENTENCE AND ALPHA VALUES
    return  pred_sent , alpha_values

In [ ]:
import nltk.translate.bleu_score as bleu

In [ ]:
BLEU = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model)[0].split()
        act = [str(i.dec_output).split()]
        b = bleu.sentence_bleu(act,pred)
        BLEU.append(b)
    except:
        print(ind)
        print(i.enc_input)
print("BELU = ", np.mean(BLEU))

## PREDICTIONS FOR TEST DATASET

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[19],model)[0])
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[19])

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[50],model)[0])
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[50])

## Inference Time

In [ ]:
%%time
predict(df_test.enc_input.values[50],model)[0]

## Beam Search BLUE Score

In [ ]:
def beam_search(input,model,k):
    seq = tk_inp.texts_to_sequences([input])
    seq = pad_sequences(seq,maxlen = 35,padding="post")

    state = model.layers[0].initialize(1)
    # GETTING THE ENCODED OUTPUT
    enc_output,enc_state_h,enc_state_c = model.layers[0](seq,state)
    

    input_state_h = enc_state_h
    input_state_c = enc_state_c 
    k_beams = [[tf.ones((1,1),dtype=tf.int32),0.0]]
    for i in range(35):
        candidates = []
        for sent_pred , prob in k_beams :
            if tk_out.word_index["<end>"] in sent_pred.numpy() :
                candidates += [[sent_pred , prob]]
            else:
               
                dec_input = model.layers[1].layers[0].layers[0](sent_pred)
                dec_output , dec_state_h , dec_state_c   =  model.layers[1].layers[0].layers[2](dec_input ,  initial_state =  [input_state_h , input_state_c])

                context_vec , alphas =  model.layers[1].layers[0].layers[1](enc_output,dec_state_h)

                # CONCATINATING THE CONTEXT VECTOR(BY EXPANDING DIMENSION) AND ENBEDDED VECTOR
                dense_input =  tf.concat([tf.expand_dims(context_vec,1),tf.expand_dims(dec_state_h,1)],axis=-1)
                
                # PASSING THE DECODER OUTPUT THROUGH DENSE LAYER WITH UNITS EQUAL TO VOCAB SIZE
                dense = model.layers[1].layers[0].layers[3](dense_input)

                pred = tf.argsort(dense, direction= 'DESCENDING')[:,:,:k]
                for w in range(k):
                  candidates += [[tf.concat((sent_pred, pred[:,:,w]) , axis=-1) , (prob + tf.math.log(dense[:,:,pred[:,:,w][0][0]])[0][0])]  ]
        k_beams = sorted(candidates,key=lambda tup:tup[1],reverse=True)[:k]

    all_sent = []
    for i,score in k_beams:
        sent = ""
        for j in range(1,35):
            sent +=  tk_out.index_word[i.numpy()[:,j][0]] +  " " 
            if tk_out.index_word[i.numpy()[:,j][0]] =="<end>":
                break
        all_sent.append((sent.strip(),score.numpy()))
    return all_sent

In [ ]:
# VALIDATION BELU SCORE
BLEU_beam = []
index = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = beam_search(str(i.enc_input),model,3)[0][0].split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_beam.append(b)
    except:
        index.append(ind)

print("BELU Score = ",np.mean(BLEU_beam))  

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[19])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[19],model,3))
for i in bm:
    print(i)

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[50])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[50],model,3))
for i in bm:
    print(i)

## Model Comparison

In [ ]:
s1 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/char_trainable_embedding/besh.h5")
s2 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_trainable_embedding/besh.h5")
s3 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_w2v/besh.h5")
s4 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_ft/besh.h5")
s5 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/bidirectional_train_emb/besh.h5")
s6 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/multi_layered_word/besh.h5")
s7 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_dot/besh.h5")
s8 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_gernal/best.h5")
s9 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_concat/best.h5")
s10 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/monitonic_attention_dot/best.h5")
s11 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/monotonic_attention_general/best.h5")
s12 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/monotonic_attention_concat/best.h5")

In [ ]:
df_comp = pd.DataFrame()
df_comp["Model"] = ["Encoder Decoder(Char Level)","Encoder Decoder","Encoder Decoder","Encoder Decoder","Bidirectional Encoder Decoder","Multilayered Encoder Decoder","Attention Dot Model","Attention Gernal Model","Attention Concat Model","Monotonic Attention Dot Model","Monotonic Attention Gernal Model","Monotonic Attention Concat Model"]
df_comp["Embedding"] = ["One Hot Encoding","Trainable Embedding" , "Pretrained Word2Vec " ,"Fast Text","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding"]
df_comp["BLEU Score(Greedy Search)"] = [0.3139,0.4603,0.4453,0.4569,0.4509,0.4527, 0.5055,0.5545,0.5388,0.5469,0.5514,0.5348]
df_comp["BLEU Score(Beam Search)"] = ["-","-","-","-",0.4561,0.4557,0.5411,0.5324,0.5671,"-","-","-"]
df_comp["Model Size(bytes)"] = [ s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12]
df_comp["Model Parameters"] = ["616,488	","26,363,578" , "8,158,378", "8,158,378","35,018,938" ," 28,464,826","33,353,914","33,419,706","33,485,755","33,353,914","33,419,706","33,485,755"]
df_comp["Inference Time(ms)"] = [143,92.3 , 94.5 , 93.7,250,311,157,164,189,164,179,176]
df_comp